## Burgers 2D Mapped Tent Pitching and Picard Propagation

In [6]:
from netgen.geom2d import SplineGeometry
from ngsolve import Mesh, L2, GridFunction, CoefficientFunction, x, y, exp
from ngsolve import TaskManager, SetNumThreads
from ngsolve.webgui import Draw
from ngstents import TentSlab
from ngstents.conslaw import Burgers
import tentswebgui

In [7]:
# SetNumThreads(1)
maxh = 0.05
geom = SplineGeometry()
geom.AddRectangle((0, 0), (1, 1), bc=1)
mesh = Mesh(geom.GenerateMesh(maxh=maxh))


In [8]:
# heapsize = 10*1000*1000
dt = 0.05   # tent slab height (in time)
wavespeed = 16
local_ctau = True
global_ctau = 1
ts = TentSlab(mesh, method="edge")
ts.SetMaxWavespeed(wavespeed)
ts.PitchTents(dt=dt, local_ct=local_ctau, global_ct=global_ctau)
print("max slope", ts.MaxSlope())
print("n tents", ts.GetNTents())

# ts = TentSlab(mesh, dt, c, heapsize)
tentswebgui.Draw(ts)

max slope 0.0625000000000002
n tents 9673
setup: 0.4548001289367676, tents: 1.0865249633789062 faces: 6.6120078563690186


NGSTentsWebGuiWidget(value={'ntents': 9673, 'nlayers': 93, 'ngsolve_version': '6.2.2101-17-g7fcf4d59d', 'tent_…

In [10]:
order = 4
cf = CoefficientFunction(exp(-50*((x-0.3)*(x-0.3)+(y-0.3)*(y-0.3))))

V = L2(mesh, order=order)
u = GridFunction(V,"u")
burg = Burgers(u, ts)
burg.SetTentSolver("SARK",substeps=order*order)
burg.SetInitial(cf)

scene = Draw(u, animate=True)

tend = 3*dt
t = 0
cnt = 0

with TaskManager():
    while t < tend - dt/2:
        burg.Propagate()
        t += dt
        cnt += 1
        if cnt % 1 == 0:
            print("{:.3f}".format(t))
            scene.Redraw()


NGSWebGuiWidget(value={'ngsolve_version': '6.2.2101-17-g7fcf4d59d', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2,…

0.050
0.100
0.150
